In [3]:
#  ************EXERCISE - 02  && 03 ****************


Ex-2
# what i have to do is
# 1. split the dataset randomly into 80% train set, 10% dev set, 10% test set. 
# 2. train the trigram model only on the training set and evaluate them on dev and test splits.
# 3. Evaluate.
# Ok 


# what is dev set?
# ok this is used during the training phase, why? -> to see if our model is overfitting or not
# if at some point our model starts to perform worst than previous on the dev set, then we can conclude
# our model has overfitted, so we can stop here. nice

# is stoping is only the choice?

# how regularization helps? our model has overfitted mean it already learned too much. i think we should stop
# why do regularization and train more?

# regularization
# -> adding penalty to the loss function to prevent it from overfitting
# -> paile nai overfitt vayeni ni feri simple working model ma lane raixa hai regularization le

# is regularization better than early stopping? if so why?


In [ ]:
Realization

i trained my trigram model many times in assignment-1 and i was suprsied why
my model is not giving good names eventhough loss was much better.

Haha it was due to overfitting. hehe
ok so i have to use early stopping technique

i will make loop 300 times and use early-stopping technique.



In [2]:
#plan

# I will train the model 2 times.
# 1st-->   I will not use early stopping technique
# 2nd-->   I will use early stopping technique

# And compare which will be better

import torch
words = open('names.txt','r').read().splitlines()

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [4]:
import torch.nn.functional as F

In [5]:
# create the dataset

xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append((ix1*27)+ix2)
    ys.append(ix3)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*27, 27), generator=g, requires_grad=True)

number of examples:  196113


In [6]:
from sklearn.model_selection import train_test_split

X_train,X_temp,y_train,y_temp = train_test_split(xs,ys,test_size = 0.2, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp,y_temp, test_size=0.5, random_state = 42)

# X_train, y_train ma 80% data xa ani
#X_dev ra X_test ma 10 10% data xa ni

In [7]:
# print(X_train.size())
# print(X_dev.size())
# print(X_test.size())

train_size = len(X_train)
dev_size = len(X_dev)
test_size = len(y_test)

In [8]:
# NOW i have to create a variable to store the dev_loss every time. if at some point dev_loss(new) > dev_loss(past) 
#stop the training

#Also let's find after how many iteration we did early stopping
#Also let's see if doing early stopping make our results better

dev_loss = 1e10

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
for k in range(1000):
    
    
    #forward pass for train-set
    
    xenc = F.one_hot(X_train, num_classes = (27*27)).float() #input to the network: one-hot encoding
    logits = xenc@W  #log-counts predict gareko hai
    counts = logits.exp()  #coutns, equivalent to N
    probs = counts/counts.sum(1,keepdim = True) #probabilities for next character
    
    loss = -probs[torch.arange(train_size),y_train].log().mean()  # (removing this, i suspect this is regularization) + 0.01 * (W**2).mean()  # i haven't understood this second part
    
    
    #forward pass for dev-set
    xenc_dev = F.one_hot(X_dev, num_classes = (27*27)).float()
    logits_dev = xenc_dev @ W
    counts_dev = logits_dev.exp()
    probs_dev = counts_dev/ counts_dev.sum(1, keepdim = True)
    loss_2 = -probs_dev[torch.arange(dev_size),y_dev].log().mean()
    
    print(f"dev_loss: {loss_2.item()}")
    
    if(dev_loss < loss_2):
        print(f"early stopping done, stoped at: {k}")
        break
    else:
        dev_loss = loss_2
    #maile bujeko yo loss chai,
#     probs[torch.arrange(num),ys] this gives probability predcited by our model for ground truth. if it is less than 
#     100% model we give it a loss i.e -loglikelihood now model updates it's parameters.
    
#     Q. I am super-amazed how this loss worked? 
#     --> because we are not always sure what letters comes after pairs in real names. any letter can come. 
#         but some are more likely.
#     --> i think our model will twiks our W such that it gives more probabilty for more likely words based on training set
    
#     Q. Still confused how loss will converged? I suspect it will not. (noob me, think later)
#     Q. 

     
    print(f"loss: {loss.item()}")
    #backward pass:
    
    W.grad = None # paile sabaile gradient lai zero gareko
    loss.backward()  #magic hehe
    
    #update
    
    W.data += -80*W.grad   #remember to increment opp to direction of gradient, otherwise face the consequences)

    

dev_loss: 2.1711983680725098
loss: 2.1655941009521484
dev_loss: 2.170900821685791
loss: 2.1652705669403076
dev_loss: 2.170604944229126
loss: 2.1649491786956787
dev_loss: 2.1703109741210938
loss: 2.1646296977996826
dev_loss: 2.1700191497802734
loss: 2.1643121242523193
dev_loss: 2.169729471206665
loss: 2.163996696472168
dev_loss: 2.1694412231445312
loss: 2.1636831760406494
dev_loss: 2.1691553592681885
loss: 2.1633718013763428
dev_loss: 2.1688711643218994
loss: 2.163062572479248
dev_loss: 2.168588876724243
loss: 2.162754535675049
dev_loss: 2.1683082580566406
loss: 2.1624491214752197
dev_loss: 2.16802978515625
loss: 2.1621451377868652
dev_loss: 2.167752742767334
loss: 2.1618435382843018
dev_loss: 2.16747784614563
loss: 2.161543369293213
dev_loss: 2.1672046184539795
loss: 2.161245107650757
dev_loss: 2.166933059692383
loss: 2.1609487533569336
dev_loss: 2.166663408279419
loss: 2.160654067993164
dev_loss: 2.166395425796509
loss: 2.1603615283966064
dev_loss: 2.1661291122436523
loss: 2.160070419

KeyboardInterrupt: 

In [ ]:
# nice, we stopped early at 50 iteration with loss 2.455
#update, many iterations with loss 2.13
# wow nice we stopped the overfitting here, Hurrehy!!!!!!


# let's see the result

In [14]:
#here loss is very good. let's test in test set

xenc_test = F.one_hot(X_test, num_classes = (27*27)).float()
logits_test = xenc_test @ W
counts_test = logits_test.exp()
probs_test = counts_test/ counts_test.sum(1, keepdim = True)
loss_3 = -probs_test[torch.arange(test_size),y_test].log().mean()

print(loss_3.item())

2.160979986190796


In [15]:
#wow loss on test set is also very good.

#which means our model is performing good.

#let's verify it by the output

In [16]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(10):

  out = []
  ix = 0
  gt = torch.randint(1,27,(1,)).item()  
  while True:

    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    ix = (ix*27) + (gt)
    out.append(itos[gt])
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27*27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------

    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    ix , gt = gt, ix
    
    if gt == 0:
      break
  print(''.join(out))

da
monde
milyasid
na
yus
ey
welin
reshir
qtonian
jayel


In [96]:
# # OK i don't see any difference. names still sucks


# Do i have to use regularizations or what?
# it is giving shitty names?


In [97]:
# wtqkwumdhntauvrfdleniajkdbyainrwibwlasnjywnfyla    wtf is this name???

# correction, my forward pass for devset was incorrect

#update: Now names are good.

In [ ]:
EX-3
What i have to do is:
    1. use dev-set to tune the strength of smoothing. 
    2. is this mean i have to find the best equation of the regularization to minimize the loss?
    3. what patterns i see in train and dev set loss as you tune this strength
    
    
My plans:
    
    1. First I will see what minimum loss i can get with early stopping
    2. I i will try different regularization to decrease the loss even further
    


minimum_loss_with_early_stopping:    